In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import json
import numpy as np
from collections import Counter
import pandas as pd
from feedback.plotting import *
from sklearn.metrics import cohen_kappa_score
from scipy.stats import pearsonr
from feedback.fr_feedback import *
from feedback.database import *


In [3]:
plt.style.use('ggplot')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.style.use('default')

In [4]:
pd.set_option('display.max_columns', None)  


In [5]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [6]:
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

plt.rcParams['savefig.bbox'] = 'tight'

In [7]:
style_size = 'paper'

In [8]:
db_name = os.environ.get("POSTGRES_DB")
username = os.environ.get("POSTGRES_USER")
password = os.environ.get("POSTGRES_PASSWORD") 
ip_address = os.environ.get("DATABASE_HOST") 
port = os.environ.get("DATABASE_PORT")
openai_api_key = os.environ.get("OPENAI_API_KEY")

connection_dict = open_connection(db_name,username,password,ip_address,port)
connection = connection_dict['connection']
cursor = connection_dict['cursor']

client = openai.OpenAI(api_key=openai_api_key)
prompt = open("../../data/prompts/update_direction.txt").read()

In [9]:
start_date = "2024-01-01"
end_date = "2024-12-31"
organization_id = 1

In [23]:
df_orig = get_feedback_by_date_instruction(connection,start_date,end_date)
df_predictions = get_predictions_by_date(connection,start_date,end_date,organization_id)
filtered_df = df_orig[df_orig['id'].isin(df_predictions.loc[df_predictions['direction_problem'] == True, 'owner_id'])]
filtered_df = filtered_df.reset_index()

/usr0/home/naveenr/projects/food_rescue_feedback/feedback/database.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [25]:
for i in range(len(filtered_df)):
    comment = {'donor': filtered_df.loc[i, "donor_name"], 
    'recipient': filtered_df.loc[i, "recipient_name"], 
    'volunteer_comment': filtered_df.loc[i, "volunteer_comment"],
    'donor_instruction': filtered_df.loc[i,"donor_instruction"], 
    'recipient_instruction': filtered_df.loc[i,"recipient_instruction"]}
    comment = str(comment)

    print("On Rescue {} out of {}".format(i+1,len(filtered_df)))

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt + comment}],
            response_format={"type": "json_object"},
        )
        output = response.choices[0].message.content
        feedback_info = json.loads(output)

        filtered_df.loc[i, 'donor_direction_change'] = feedback_info['donor_direction_change']
        filtered_df.loc[i, 'rewritten_donor_direction'] = feedback_info['rewritten_donor_direction']
        filtered_df.loc[i, 'recipient_direction_change'] = feedback_info['recipient_direction_change']
        filtered_df.loc[i, 'rewritten_recipient_direction'] = feedback_info['rewritten_recipient_direction']

    except Exception as e:
        print(f"Error processing feedback {i}: {e}")

On Rescue 1 out of 130
On Rescue 2 out of 130
On Rescue 3 out of 130
On Rescue 4 out of 130
On Rescue 5 out of 130
On Rescue 6 out of 130
On Rescue 7 out of 130
On Rescue 8 out of 130
On Rescue 9 out of 130
On Rescue 10 out of 130
On Rescue 11 out of 130
On Rescue 12 out of 130
On Rescue 13 out of 130
On Rescue 14 out of 130
On Rescue 15 out of 130
On Rescue 16 out of 130
On Rescue 17 out of 130
On Rescue 18 out of 130
On Rescue 19 out of 130
On Rescue 20 out of 130
On Rescue 21 out of 130
On Rescue 22 out of 130
On Rescue 23 out of 130
On Rescue 24 out of 130
On Rescue 25 out of 130
On Rescue 26 out of 130
On Rescue 27 out of 130
On Rescue 28 out of 130
On Rescue 29 out of 130
On Rescue 30 out of 130
On Rescue 31 out of 130
On Rescue 32 out of 130
On Rescue 33 out of 130
On Rescue 34 out of 130
On Rescue 35 out of 130
On Rescue 36 out of 130
On Rescue 37 out of 130
On Rescue 38 out of 130
On Rescue 39 out of 130
On Rescue 40 out of 130
On Rescue 41 out of 130
On Rescue 42 out of 130
O

In [61]:
filtered_df = filtered_df[
    (filtered_df['donor_direction_change'] == True) |
    (filtered_df['recipient_direction_change'] == True)
]


In [59]:
filtered_df.to_csv("../../data/annotations/rewritten_instructions_2024.csv")